In [ ]:
import pandas as pd
import pm4py
import numpy as np


In [ ]:
# hinge = pm4py.read_ocel2("socel_hinge.xml")

In [ ]:
events = pd.read_csv('events.csv', keep_default_na=False) # keep_default_na=False
for col in events.columns:
    if col in ["ocel:timestamp", "ocel:eid", "ocel:activity", "paint name"]:
        continue
    events[col] = pd.to_numeric(events[col])
events = events.fillna(np.nan)

events["ocel:timestamp"] = pd.to_datetime(events["ocel:timestamp"], format='%d.%m.%Y %H:%M:%S')
objects = pd.read_csv('objects.csv', keep_default_na=False) # keep_default_na=False
for col in objects.columns:
    if col in ["ocel:oid", "ocel:type", "type", "recycled"]:
        continue
    objects[col] = pd.to_numeric(objects[col])
objects = objects.fillna(np.nan)
e2o = pd.read_csv('e2o.csv', keep_default_na=False) # keep_default_na=False
e2o["ocel:timestamp"] = pd.to_datetime(e2o["ocel:timestamp"], format='%d.%m.%Y %H:%M:%S')

In [ ]:
from pm4py.objects.ocel.obj import OCEL
ocel = OCEL(objects=objects, events=events, relations=e2o)

In [ ]:
ocpn = pm4py.ocel.discover_oc_petri_net(ocel)
pm4py.vis.view_ocpn(ocpn)

In [ ]:
ocdfg = pm4py.ocel.discover_ocdfg(ocel)
pm4py.vis.view_ocdfg(ocdfg, annotation="performance")

In [ ]:
pm4py.vis.save_vis_ocdfg(ocdfg, "ocdfg.svg", format="svg", annotation="performance")

In [ ]:
pm4py.write_ocel2_sqlite(ocel, "ocel")
pm4py.write_ocel2_xml(ocel, "ocel")
pm4py.write_ocel2_json(ocel, "ocel")

In [ ]:
# load impact overview
impacts = pd.read_csv('impacts.csv', keep_default_na=False)
ibjects = impacts[impacts["type type"] == "object"]
ivents = impacts[impacts["type type"] == "event"]

In [ ]:
CHART_COLOURS = ["#006165", # petrol
                 "#CE108A", # pink
                 "#0098A1", # turquoise
                 "#F6A800", # orange
                "#00549F", # blue
                "#6f2b4b",# purple
                 "#8EBAE5", # light blue
                "#000080", # dark blue
                "#007e56", # lighter greeny-turquoise
                "#005d4c", # perl-ophal green
                "#a1dfd7", #light-turquoise
                "#cd00cd", # pink

                 "#28713e", # green
                 "#701f29", # purpur-red

                 "#5d2141", # other purple
                 "#a1dfd7", #light-turquoise

                "#00ffff", # cyan
                "#39ff14", # neon green
                 "#800080", #purpur
                "#005f6a", # blue-petrol
                "#76e1e0", # another turquopise
                "#f5ff00" # neon-yellow
                 ]

In [ ]:
import plotly.express as px
fig = px.sunburst(ibjects, path=['type type', 'instance type', 'instance', 'type'], values='impact', color='type', color_discrete_sequence=CHART_COLOURS)
# show colors in legend


fig.show()

In [ ]:
ivents = ivents.sort_values(by='impact', ascending=False)
fig = px.sunburst(ivents, path=['type type', 'instance type', 'instance', 'name'], values='impact', color='type', color_discrete_sequence=CHART_COLOURS)
fig.show()

In [ ]:
# allocated impacts
alloc = pd.read_csv('allocated_impacts.csv')

In [ ]:
# create stacked bar chart for allocated impacts per object
import plotly.express as px
fig = px.bar(alloc, x="bike", y="impact", pattern_shape="type", color="name", barmode="stack", color_discrete_sequence=CHART_COLOURS)
fig.show()

In [ ]:
import pygraphviz as pgv
G = pgv.AGraph(directed=False)
G.add_node("bk1", color="#CE108A")
G.add_node("bk2", color="#CE108A")
# G.add_node("f1", color="#0098A1")
# G.add_node("f2", color="#0098A1")
# G.add_node("w1", color="#0098A1")
# G.add_node("w2", color="#0098A1")
# G.add_node("g1", color="#0098A1")
# G.add_node("g2", color="#0098A1")
G.add_nodes_from(["o1", "o2", "sm1", "sm2", "dc1", "t1", "pk1", "g1", "g2", "f1", "f2", "w1", "w2"])

In [ ]:
edges = [
    ("o1", "w1"),
    ("o1", "f1"),
    ("o1", "g1"),
    ("w1", "f1"),
    ("f1", "g1"),
    ("f1", "bk1"),
    ("bk1", "g1"),
    ("bk1", "t1"),
    ("bk1", "pk1"),
    ("w1", "g1"),
    ("f1", "sm1"),
    ("f1", "dc1"),
    ("f2", "sm2"),
    ("f2", "dc1"),
    ("f2", "bk2"),
    ("f2", "o2"),
    ("f2", "w2"),
    ("f2", "g2"),
    ("w2", "g2"),
    ("w2", "o2"),
    ("w2", "bk2"),
    ("bk2", "t1"),
    ("bk2", "pk1"),
    ("bk2", "g2"),
    ("o2", "g2"),
]
G.add_edges_from(edges)

In [ ]:
G.layout(prog="dot")
G.draw("object_graph.png")